## Data Description

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [12]:
%%sql
Select *
from genre
group by genre_id;

Done.


genre_id,name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll
6,Blues
7,Latin
8,Reggae
9,Pop
10,Soundtrack


In [16]:
%%sql
select *
from invoice_line
limit 2;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1


In [149]:
%%sql
select *
from invoice
limit 2;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9


## New Albums Selection

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce.

In [45]:
%%sql

WITH usa_track_sold AS (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    WHERE i.billing_country = "USA")
SELECT 
    g.name AS genre, count(uts.invoice_line_id) as "Number of tracks sold", cast(count(uts.invoice_line_id)as float)/(select count(*) from usa_track_sold) as "Percentage_sold"
    from usa_track_sold uts inner join track t on uts.track_id = t.track_id
    inner join genre g on t.genre_id = g.genre_id
    group by g.genre_id
    order by 2 desc;

Done.


genre,Number of tracks sold,Percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Accoridng to above analysis, we should purchase albums from 3 artists as follow:
- Red Tone(Punk)
- Slim Jim Bites(Blues)
- Meteor and the Girls(Pop)

However, those genres only account 17% of sales. Chinook should invest more on Rock genre, which makes up 53% of sales in the store.

## Employee performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [51]:
%%sql
select * from customer limit 2

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5


In [81]:
%%sql
select * from employee 

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [66]:
%%sql
SELECT e.last_name || " " || e.first_name AS Employee, e.hire_date, SUM(i.total) AS "Sales"
FROM employee e INNER JOIN customer c on e.employee_id = c.support_rep_id
                INNER JOIN invoice i on i.customer_id = c.customer_id
GROUP BY 1    
ORDER BY 2 DESC;

Done.


Employee,hire_date,Sales
Johnson Steve,2017-10-17 00:00:00,1393.920000000002
Park Margaret,2017-05-03 00:00:00,1584.0000000000034
Peacock Jane,2017-04-01 00:00:00,1731.5100000000039


In [76]:
%%sql
SELECT e.last_name || " " || e.first_name AS Employee, e.hire_date, CAST ((julianday('now') - julianday(e.hire_date)) / 30 AS integer) as "Tenure(months)", 
       SUM(i.total) AS "Total Sales", AVG(i.total) as "Avg Sales/month"
FROM employee e INNER JOIN customer c on e.employee_id = c.support_rep_id
                INNER JOIN invoice i on i.customer_id = c.customer_id
GROUP BY 1    
ORDER BY 2 DESC;

Done.


Employee,hire_date,Tenure(months),Total Sales,Avg Sales/month
Johnson Steve,2017-10-17 00:00:00,38,1393.920000000002,7.414468085106393
Park Margaret,2017-05-03 00:00:00,44,1584.0000000000034,7.401869158878521
Peacock Jane,2017-04-01 00:00:00,45,1731.5100000000039,8.167500000000018


In [97]:
%%sql
SELECT e.first_name || " " || e.last_name AS Employee, e.hire_date, CAST ((julianday('now') - julianday(e.hire_date)) / 30 AS integer) as "Tenure_in_month", 
       SUM(i.total) AS "Total_Sales", SUM(i.total)/CAST ((julianday('now') - julianday(e.hire_date)) / 30 AS integer) as "Avg Sales/month"
FROM employee e INNER JOIN customer c on e.employee_id = c.support_rep_id
                INNER JOIN invoice i on i.customer_id = c.customer_id
GROUP BY 1   
ORDER BY 4 DESC;

Done.


Employee,hire_date,Tenure_in_month,Total_Sales,Avg Sales/month
Jane Peacock,2017-04-01 00:00:00,45,1731.5100000000039,38.47800000000009
Margaret Park,2017-05-03 00:00:00,44,1584.0000000000034,36.00000000000008
Steve Johnson,2017-10-17 00:00:00,38,1393.920000000002,36.68210526315794


In [89]:
%%sql
SELECT e.first_name || " " || e.last_name AS Employee, SUM(i.total) AS "Total Sales"
FROM employee e INNER JOIN customer c on e.employee_id = c.support_rep_id
                INNER JOIN invoice i on i.customer_id = c.customer_id
WHERE strftime('%Y',i.invoice_date) = '2020'
GROUP BY 1   
ORDER BY 2 DESC;

Done.


Employee,Total Sales
Jane Peacock,401.93999999999994
Steve Johnson,371.25000000000006
Margaret Park,365.31


While Jane performed the best with sales of \$1,731 during her tenure, Steve has lowest sales of \$1,393. However, Steve joined later than Jan so I calculate the monthly sales to have more accurate evaluation. As you see, Jane still generated the most sale but Steve performed slightly better than Margaret. This difference may be due to the sales from year 2020, the next table clearly shows that Steve has higher sale than Margaret.

## Sales by Country

Your next task is to analyze the sales data for customers from each different country.

In [100]:
%%sql
select * from invoice limit 2

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9


In [161]:
%%sql
WITH country_group AS (
SELECT billing_country, count(distinct customer_id) customer_count,
     CASE
         WHEN count(distinct customer_id) = 1 THEN 'Other'
         ELSE billing_country
     END as Country
 FROM invoice
 GROUP BY billing_country
 ORDER BY 2 DESC)
SELECT cg.Country, count(distinct i.customer_id) "Total_customer", SUM(i.total) "Total_Sales", 
        sum(i.total)/count(distinct i.customer_id) "Sales_per_customer", sum(i.total)/count(distinct i.invoice_id) "avg_order",
        CASE
            WHEN cg.Country = "Other" THEN 1
            ELSE 0
        END AS sort
FROM invoice i INNER JOIN customer c ON i.customer_id = c.customer_id
               INNER JOIN country_group cg on i.billing_country = cg.billing_country
Group by 1
Order by sort, Total_Sales desc;

Done.


Country,Total_customer,Total_Sales,Sales_per_customer,avg_order,sort
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189,0
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265,0
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075,0
France,5,389.0699999999999,77.81399999999998,7.781399999999998,0
Germany,4,334.62,83.655,8.161463414634147,0
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002,0
United Kingdom,3,245.52,81.84,8.768571428571429,0
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276,0
India,2,183.14999999999998,91.57499999999999,8.72142857142857,0
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143,1


While the store has attracted most customers from USA and Canada, there is a growth opportunity in these countries:
- Czech Republic
- United Kingdom
- India
However, because the sample size is really low, we should do market reasearch before investing marketing campaign to those areas. 

## Albums vs Individual Tracks

Management are currently considering changing their purchasing strategy to save money. They have two options as follow:
- purchase a whole album
- purchase a collection of one or more individual tracks

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [162]:
%%sql
select *
from album
limit 2

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2


In [167]:
%%sql
select *
from invoice
limit 2

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9


In [195]:
%%sql
select *
from invoice_line
limit 2

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1


In [166]:
%%sql
select track_id, album_id
from track
limit 2

Done.


track_id,album_id
1,1
6,1


In [202]:
%%sql
WITH invoice_album AS(
SELECT i.invoice_id, album_id
FROM invoice_line i INNER JOIN track t on i.track_id = t.track_id
GROUP BY invoice_id, album_id)
SELECT Same_album, count(invoice_id) "total_invoices", CAST(count(invoice_id) AS FLOAT)/(SELECT count(*) FROM invoice) "percent"
FROM (SELECT invoice_id, count(distinct album_id) 'total_album', CASE
        WHEN count(distinct album_id) = 1 THEN 'Yes'
        ELSE 'No'
        END as 'Same_album'
FROM invoice_album
GROUP BY invoice_id)
GROUP BY 1

Done.


Same_album,total_invoices,percent
No,443,0.7214983713355049
Yes,171,0.2785016286644951


Based on the analysis, while 72% of customers prefer to buy tracks from multiple albums, only 28% of them purchase whole album at once. Therefore, I would recommend not continue to buy full albums from record companies.